In [126]:
import pandas as pd 
import numpy as np 
import seaborn as sns
import matplotlib.pyplot as plt

In [127]:
df = pd.read_csv("data/Train.csv")
#df.head()

In [128]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 15539 entries, 0 to 15538
Data columns (total 9 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   ID            15539 non-null  object 
 1   location      15539 non-null  object 
 2   temp          15539 non-null  object 
 3   precip        15539 non-null  object 
 4   rel_humidity  15539 non-null  object 
 5   wind_dir      15539 non-null  object 
 6   wind_spd      15539 non-null  object 
 7   atmos_press   15539 non-null  object 
 8   target        15539 non-null  float64
dtypes: float64(1), object(8)
memory usage: 1.1+ MB


# basic cleanup and conversion

In [129]:
# covert features  fron string to List of values 
def replace_nan(x):
    if x==" ":
        return np.nan
    else :
        return float(x)
features=["temp","precip","rel_humidity","wind_dir","wind_spd","atmos_press"]
for feature in features : 
    df[feature] = df[feature].apply(lambda x: [ replace_nan(X) for X in x.replace("nan"," ").split(",")])

# drop rows with too many NaNs

In [130]:
# recording periods contain varying degrees of NaNs: compute percent NaN for each recording period and feature
# function to compute the percentage of NaNs per recording period
def compute_percent_nan(df, col_name):
    df['percent_nan_'+col_name] = df[col_name].apply(lambda x: np.isnan(np.array(x)).sum()/len(x)*100)
    return df

In [131]:
# calculate percentage of missings per recording period and feature and append to dataframe
for col_name in features:
    data=compute_percent_nan(df,col_name)

In [132]:
#df.head()

In [133]:
# filter observations based on percent NaN and check again the data distribution of the target and summary features
nan_columns = ['percent_nan_temp', 'percent_nan_precip','percent_nan_rel_humidity', 'percent_nan_wind_dir','percent_nan_wind_spd', 
        'percent_nan_atmos_press']
df = df[(df[nan_columns]<30).all(axis=1)]
df.drop(nan_columns, axis=1, inplace=True)
df.reset_index(inplace=True)
df.head()

/Users/alwinklick/neuefische/ds-air-quality/.venv/lib/python3.9/site-packages/pandas/core/frame.py:4906: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return super().drop(


,index,ID,location,temp,precip,rel_humidity,wind_dir,wind_spd,atmos_press,target
0,1,ID_train_1,D,"[22.53333333, 21.71666667, 20.83333333, 20.983...","[0.102, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.034, 0...","[0.744583333, 0.808083333, 0.911166667, 0.9163...","[281.6643101, 89.15629262, 81.96853891, 291.01...","[2.3775, 1.126666667, 0.700833333, 0.341666667...","[90.32, 90.3775, 90.44083333, 90.4725, 90.4541...",79.131702
1,5,ID_train_10000,E,"[26.225, 26.25, 26.95833333, 27.925, 28.416666...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[0.7566666670000001, 0.7090833329999999, 0.632...","[nan, 104.3737208, 161.28512859999995, 161.322...","[nan, 1.4858333330000002, 1.090833333, 0.94166...","[88.56583333, 88.55, 88.50916667, 88.44083333,...",17.005000
2,6,ID_train_10001,C,"[21.86666667, 21.38333333, 20.75, 20.74166667,...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[0.548333333, 0.571666667, 0.589166667, 0.57, ...","[334.0827161, 332.4037603, 328.58290530000005,...","[1.1525, 1.055, 1.11, 1.385833333, 1.2, 1.2508...","[87.73666667, 87.74416667, 87.7475, 87.78, 87....",53.100000
3,7,ID_train_10002,A,"[28.24166667, 27.9, 26.45833333, 24.31666667, ...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[0.52725, 0.515833333, 0.585916667, 0.6965, 0....","[328.4616193, 336.5790368000001, 346.5753952, ...","[1.926666667, 1.635, 0.744166667, 0.454166667,...","[88.01416667, 88.01166667, 88.0375, 88.1308333...",55.051143
4,10,ID_train_10005,D,"[19.4, 18.99166667, 18.56666667, 18.24166667, ...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[0.9275, 0.93, 0.935833333, 0.936666667, 0.935...","[30.34376711, 10.47925914, 316.7126711, 295.14...","[0.348333333, 0.555, 0.423333333, 0.320833333,...","[90.78833333, 90.72, 90.67333333, 90.65333333,...",55.638261


# do the wind thing


In [134]:

df["wind_dir_x"] = df.wind_dir.apply(lambda x: list(np.cos(np.array(x) * np.pi /180)))
df["wind_dir_y"] = df.wind_dir.apply(lambda x: list(np.sin(np.array(x) * np.pi /180)))
df.drop("wind_dir", axis=1, inplace=True)
features=["temp","precip","rel_humidity","wind_dir_x","wind_dir_y","wind_spd","atmos_press"]

df.head(3)

/var/folders/3x/gn1x_n556cx49y1dx9kzy3mm0000gn/T/ipykernel_55398/1058124178.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df["wind_dir_x"] = df.wind_dir.apply(lambda x: list(np.cos(np.array(x) * np.pi /180)))
/var/folders/3x/gn1x_n556cx49y1dx9kzy3mm0000gn/T/ipykernel_55398/1058124178.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df["wind_dir_y"] = df.wind_dir.apply(lambda x: list(np.sin(np.array(x) * np.pi /180)))
/Users/alwinklick/neuefische/ds-air-quality/.venv/lib/python3.9/site-packages/pand

,index,ID,location,temp,precip,rel_humidity,wind_spd,atmos_press,target,wind_dir_x,wind_dir_y
0,1,ID_train_1,D,"[22.53333333, 21.71666667, 20.83333333, 20.983...","[0.102, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.034, 0...","[0.744583333, 0.808083333, 0.911166667, 0.9163...","[2.3775, 1.126666667, 0.700833333, 0.341666667...","[90.32, 90.3775, 90.44083333, 90.4725, 90.4541...",79.131702,"[0.20217729203070506, 0.014724939532783088, 0....","[-0.9793489381150781, 0.9998915822006683, 0.99..."
1,5,ID_train_10000,E,"[26.225, 26.25, 26.95833333, 27.925, 28.416666...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[0.7566666670000001, 0.7090833329999999, 0.632...","[nan, 1.4858333330000002, 1.090833333, 0.94166...","[88.56583333, 88.55, 88.50916667, 88.44083333,...",17.005000,"[nan, -0.2482456120597107, -0.9471270291698715...","[nan, 0.9686971229920628, 0.32085883284686667,..."
2,6,ID_train_10001,C,"[21.86666667, 21.38333333, 20.75, 20.74166667,...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[0.548333333, 0.571666667, 0.589166667, 0.57, ...","[1.1525, 1.055, 1.11, 1.385833333, 1.2, 1.2508...","[87.73666667, 87.74416667, 87.7475, 87.78, 87....",53.100000,"[0.8994259719796608, 0.8862339832634597, 0.853...","[-0.43707312995475084, -0.46323787292165763, -..."


# get last measurement from each time series

In [124]:
# get last value for each time series
# did this before dropping all NaNs


def last(x):
    """return last value of list, if not there then return second to last, if not there the one 24h before last. 

    Args:
        x (_type_): _description_

    Returns:
        _type_: _description_
    """
    if not pd.isna(x[-1]):
        return x[-1]
    if pd.isna(x[-1]) and not pd.isna(x[-2]):
        return x[-2]
    return x[-25]
    #e=1
    #while pd.isna(x[-e]) and e < len(x):
    #    e += 24
    #return x[-e]

    
def get_last(x,col_name):
    x["last_"+col_name]=x[col_name].apply(last)
    return x 

for col_name in features:
    df=get_last(df,col_name)

/var/folders/3x/gn1x_n556cx49y1dx9kzy3mm0000gn/T/ipykernel_55398/1204741954.py:26: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  x["last_"+col_name]=x[col_name].apply(last)


KeyError: 'wind_dir'

In [ ]:
df.head(3)

In [ ]:
df.info()

# other aggregation stuff

In [ ]:
# aggregation function extracting summary statistics from every recording period and appending it as a new column to a dataframe
def aggregate_features(x,col_name):
    #x["max_"+col_name]=x[col_name].apply(np.max)
    #x["min_"+col_name]=x[col_name].apply(np.min)
    x["mean_"+col_name]=x[col_name].apply(np.mean)
    x["std_"+col_name]=x[col_name].apply(np.std)
    #x["var_"+col_name]=x[col_name].apply(np.var)
    #x["median_"+col_name]=x[col_name].apply(np.median)
    #x["ptp_"+col_name]=x[col_name].apply(np.ptp)
    return x  

# function returning only non-Null values (helper for aggregation function)
def remove_nan_values(x):
    return [e for e in x if not pd.isna(e)]


In [ ]:
# remove NaNs from dataframe
for col_name in features:
   df[col_name]=df[col_name].apply(remove_nan_values)

In [ ]:
#extract summary statistics for each recording period and feature
for col_name in features:
    df=aggregate_features(df,col_name)

In [ ]:
df.drop(features, axis=1, inplace=True)

In [ ]:
df.head(3)

In [ ]:
df.dropna(inplace=True)

In [ ]:
df.info()

# claculate a quick model
without the last column, to plot residuals over it

In [ ]:
# import libraries and metrics
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LinearRegression
from sklearn.metrics import r2_score, mean_squared_error

In [ ]:
df.columns

In [ ]:
# get target and features
X = df[['last_temp', 'last_precip', 'last_rel_humidity', 'last_wind_dir', 'last_wind_spd',
       'last_atmos_press','location', 'mean_temp', 'mean_precip', 'mean_rel_humidity', 'mean_wind_dir', 'mean_wind_spd', 'mean_atmos_press']]
y = df.target

print(X.shape)
print(y.shape)

In [ ]:
# perform train test split, stratified by location in order to ensure that locations are balanced between training and test set
rseed = 42

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, stratify=X['location'], random_state=rseed)

print(X_train.shape)
print(X_test.shape)

In [ ]:
# scale features using z-transformation
scaler = StandardScaler()

# fit_transform training data, drop location column since it will not be used for prediction
X_train_scaled = scaler.fit_transform(X_train.drop('location', axis=1))
# aply transform to test data
X_test_scaled = scaler.transform(X_test.drop('location', axis=1))

In [ ]:
# train the model
linreg = LinearRegression()

linreg.fit(X_train_scaled, y_train)

In [ ]:
# predict new cases
y_pred = linreg.predict(X_test_scaled)

# show first 10 predictions
y_pred[:10]

In [ ]:
# evaluate model accurray

rmse_linreg = mean_squared_error(y_test, y_pred, squared=False)
r2_linreg = r2_score(y_test, y_pred)

print(f'RMSE on testset: {round(rmse_linreg,2)}')
print(f'Coefficient of determination on testset: {round(r2_linreg,2)}')

In [ ]:
# calculate residuals
residual = y_test - y_pred

In [ ]:
sns.scatterplot(x=X_test.last_temp, y=residual, hue=X_test['location'])
plt.xlabel('last temperature')
plt.ylabel('residual')
plt.title('Residual plot');